In [1]:
import pyodbc
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import getpass
import plotly
import plotly.express as px
import plotly.io as pio

import matplotlib.pyplot as plt
%matplotlib inline

start = dt.now()
today_date = dt.today().date()
tm_day_one = dt.today().date().replace(day=1)
tmly_day_one = tm_day_one - relativedelta(months=11)

replenish = tm_day_one - relativedelta(months=2)

print(start)
pd.set_option('display.max_columns', None)

2024-08-15 08:11:39.182428


In [2]:
# SQL python connection to our server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

# Amazon PO dataframes.

In [3]:
df_po = pd.read_csv(r'G:\SALES\Amazon\PURCHASE ORDERS\atelier\po_analysis\PurchaseOrderItems.csv'
                    ,dtype={
                        'External ID': 'object'
                        ,'Total accepted cost':'float'}
                     ,parse_dates=['Order Date']
                     )

In [4]:
df_po.columns

Index(['PO', 'Product title', 'Vendor Code', 'ASIN', 'Order Date',
       'External Id Type', 'External ID', 'Model Number', 'Merchant Sku',
       'Catalog Number', 'Requested quantity', 'Accepted Quantity',
       'ASN quantity', 'Received quantity', 'Remaining quantity', 'Cost price',
       'List price', 'Discount(%)', 'Case Size', 'Total accepted cost',
       'Cancelled quantity', 'Cancellation reason', 'Currency Code',
       'Ship to location', 'Window Start', 'Window End', 'Expected Date',
       'Freight Terms', 'Backordered', 'Cancellation deadline'],
      dtype='object')

In [5]:
df_po['Order Date'].unique()

array(['2024-08-14T00:00:00.000000000', '2024-08-13T00:00:00.000000000',
       '2024-08-12T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
# df_po = df_po.loc[df_po['Order Date'] != '45237']

In [7]:
# Summary
tc = sum(df_po['Total accepted cost'])
print(f'There are {df_po.shape[0]} rows and the total cost is ${tc:,.0f}.')
# print(tc)

There are 8947 rows and the total cost is $588,745.


In [8]:
df_po.head()

,PO,Product title,Vendor Code,ASIN,Order Date,External Id Type,External ID,Model Number,Merchant Sku,Catalog Number,Requested quantity,Accepted Quantity,ASN quantity,Received quantity,Remaining quantity,Cost price,List price,Discount(%),Case Size,Total accepted cost,Cancelled quantity,Cancellation reason,Currency Code,Ship to location,Window Start,Window End,Expected Date,Freight Terms,Backordered,Cancellation deadline
0,7ULXGNXG,Baby Triceratops,CHRON,B0CPDM8ZS4,2024-08-14,EAN,9798889880110,NaN,NaN,NaN,1,1,0,0,1,4.60,9.99,54.0,1,4.60,0,NaN,USD,"SMF3 - Stockton, CA",08/15/2024,08/20/2024,08/26/2024,Ship (Collect),NO,09/04/2024
1,7ULXGNXG,Baby Diplodocus,CHRON,B0CPDM6MT2,2024-08-14,EAN,9798889880103,NaN,NaN,NaN,2,2,0,0,2,4.60,9.99,54.0,1,9.20,0,NaN,USD,"SMF3 - Stockton, CA",08/15/2024,08/20/2024,08/26/2024,Ship (Collect),NO,09/04/2024
2,7ULXGNXG,Katie Ledecky (Spanish Edition),CHRON,B0CPDLKFDX,2024-08-14,EAN,9798892000161,NaN,NaN,NaN,1,1,0,0,1,5.06,10.99,54.0,1,5.06,0,NaN,USD,"SMF3 - Stockton, CA",08/15/2024,08/20/2024,08/26/2024,Ship (Collect),NO,09/04/2024
3,7ULXGNXG,Peekaboo Who?,CHRON,B0C9ZH5WHQ,2024-08-14,EAN,9791027612192,NaN,NaN,NaN,2,2,0,0,2,6.90,14.99,54.0,1,13.80,0,NaN,USD,"SMF3 - Stockton, CA",08/15/2024,08/20/2024,08/24/2024,Ship (Collect),NO,09/04/2024
4,7ULXGNXG,Ultimate Spotlight: Home Construction,CHRON,B0C9ZBV5WF,2024-08-14,EAN,9791027612208,NaN,NaN,NaN,8,8,0,0,8,8.74,18.99,54.0,1,69.92,0,NaN,USD,"SMF3 - Stockton, CA",08/15/2024,08/20/2024,08/26/2024,Ship (Collect),NO,09/04/2024


In [9]:
# Aggregate the po doc so we have one ISBN per line
def po_clean(df):
    df = df.groupby(by=['ASIN','External ID']).agg(
        {'Accepted Quantity':'sum','Requested quantity':'sum','Total accepted cost':'sum','Cost price':'mean'}).reset_index()
    df.rename(columns = {'External ID':'ISBN'},inplace = True)
    df['ISBN'] = df['ISBN'].str.zfill(13)
    df['ASIN'] = df['ASIN'].str.zfill(10)
    return df

In [10]:
# Clean PO table
df_po_clean = po_clean(df_po)
df_po_clean.head()

,ASIN,ISBN,Accepted Quantity,Requested quantity,Total accepted cost,Cost price
0,0473173891,9780473173890,1,1,41.38,41.38
1,0735329141,9780735329140,4,4,27.60,6.90
2,0735330972,9780735330979,18,18,99.00,5.50
3,073533112X,9780735331129,36,36,198.00,5.50
4,073533417X,9780735334175,4,4,30.00,7.50


In [11]:
df_po_clean.dtypes

ASIN                    object
ISBN                    object
Accepted Quantity        int64
Requested quantity       int64
Total accepted cost    float64
Cost price             float64
dtype: object

In [12]:
# Checking totals
print(round(sum(df_po_clean['Total accepted cost'])) == round(tc))
print(f'Total Cost: {tc:,.0f}')

True
Total Cost: 588,745


In [13]:
df_po_clean.isnull().sum()

ASIN                   0
ISBN                   0
Accepted Quantity      0
Requested quantity     0
Total accepted cost    0
Cost price             0
dtype: int64

In [14]:
df_po.shape

(8947, 30)

# Category info from the Product Catalog

In [15]:
df_cat = pd.read_csv(
                    filepath_or_buffer=r'G:\SALES\Amazon\RBH\DOWNLOADED_FILES\Catalog_Manufacturing_UnitedStates.csv'
                    ,skiprows=1
                    ,usecols=['EAN','ASIN','ISBN-13','Product Group']
                    ,na_values=['UNKNOWN','—']
                    ,dtype={'EAN': 'object','ISBN-13':'object'}
                    )

In [16]:
df_cat.columns

Index(['ASIN', 'EAN', 'ISBN-13', 'Product Group'], dtype='object')

In [17]:
def cat_clean(df):
    # remove where both EAN and ISBN-13 are both null
    df = df.copy()
    filt1 = (df_cat['EAN'].isnull()) & (df_cat['ISBN-13'].isnull())
    df = df.loc[~filt1]
    
#     # remove where list price = zero
#     filt2 = df_cat['List Price'] == '$0.00'
#     df = df.loc[~filt2]
    
#     # drop list price column
#     df.drop(['List Price'], axis=1,inplace=True)
    
    # Renaming the ean to ISBN
    df.rename(columns={"EAN": "ISBN"}, errors="raise",inplace = True)
    
    # Sometimes there is an ISBN-13 but no EAN, so i'll take the ISBN-13 if EAN is missing
    df['ISBN'] = np.where(df['ISBN'].isna(),df['ISBN-13'],df['ISBN'])
    
    # Removing the ISBN-13 since no longer needed
    df.drop(['ISBN-13'], axis=1,inplace=True)
    
    df['ISBN'] = df['ISBN'].str.zfill(13)
    df['ASIN'] = df['ASIN'].str.zfill(10)
    
    return df

In [18]:
# applying function
df_cat_clean = cat_clean(df_cat)

In [19]:
print(f'The clean cat table has {df_cat_clean.shape[0]} rows.')
print(f'The ASIN column as {df_cat_clean.ASIN.nunique()} unique values.')

The clean cat table has 29166 rows.
The ASIN column as 29128 unique values.


In [20]:
df_cat_clean.loc[df_cat_clean.duplicated(subset=['ISBN'])].sort_values(by='ISBN')

,ASIN,ISBN,Product Group
17978,1452164150,0492019587502,Books-Local Books
23634,1452149577,0499995260320,Books-Trade
22435,0811843513,0765145111249,Books-Trade
22564,0735355304,0765145122429,Toys
23818,0735348200,0765145122504,Toys
...,...,...,...
9938,B09L77MMP1,9781856699495,Books-Trade
13207,0898127432,9781926722252,Books-Trade
14005,9063692072,9789063692070,Books-Trade
13949,9063692870,9789063692872,Books-Trade


In [21]:
df_cat_clean.loc[df_cat_clean.ISBN =='9780811806763']

,ASIN,ISBN,Product Group
10718,0811806766,9780811806763,Books-Trade
11582,B004VS7D9S,9780811806763,Digital_Ebook_Purchase


In [22]:
df_cat_clean = df_cat_clean.drop_duplicates(subset='ISBN', keep="first")

# Inventory Data from the Product Catalog

In [23]:
df_avail = pd.read_excel(r'G:\OPS\Inventory\Daily\Finance_Only\Inventory Detail.xlsx'
                        ,usecols=['ISBN','Available To Sell','On Sale Date','Reprint Due Date','Reprint Quantity']
                        ,dtype={'ISBN': 'object'}
                        ,engine='openpyxl')

In [24]:
# Function to clean up the available table
# (1) removes null ISBNs, (2) makes ISBN's 13 digits long, (3) puts 0's in the empty "Reprint Quantity" bucket.

def avail_clean(df):
    df = df.loc[~df.ISBN.isnull()].reset_index(drop=True)
    df['ISBN'] = df['ISBN'].astype(str).apply(lambda x: x.zfill(13))
    df['Reprint Quantity'] = df['Reprint Quantity'].fillna(0) 
    df['ISBN'] = df['ISBN'].str.zfill(13)
    return df

In [25]:
# Clean up df_avail table from the Inventory Detail table
df_avail_clean = avail_clean(df_avail)

df_avail_clean.head()

,ISBN,On Sale Date,Reprint Due Date,Reprint Quantity,Available To Sell
0,0030955767857,NaT,NaT,0.0,1884.0
1,0030955767871,NaT,NaT,0.0,3243.0
2,0736313543308,NaT,NaT,0.0,5416.0
3,0736313543377,NaT,NaT,0.0,1650.0
4,0736313543384,NaT,NaT,0.0,2672.0


In [26]:
# There are no duplicate ISBN's in table
print(df_avail_clean.shape[0] == df_avail_clean.ISBN.nunique())
print(df_avail_clean.shape[0])

True
39160


# YPTICOD data

In [27]:
df_asin = pd.read_excel(r'\\sierra\groups\READTHIS\FINANCE\SALES\23sales\Oracle YPTICOD.xlsx'
                        ,usecols=['ISBN','ISBN10']
                        ,dtype={'ISBN': 'object','ISBN10':'object'}
                        ,engine='openpyxl')

In [28]:
def clean_ypticod(df):
    df['ISBN'] = df.loc[:,'ISBN'].astype(str).str.zfill(13)
    df['ISBN10'] = df.loc[:,'ISBN10'].astype(str).str.zfill(10)
    filt = df['ISBN10'].isna()
    df = df.loc[~filt]
    df.rename(columns = {'ISBN10':'ASIN'}, inplace = True)
    return df

In [29]:
# Applying the function
df_asin_clean = clean_ypticod(df_asin)

# Item Table Information

In [30]:
query_item = f'''
SELECT
    i.PUBLISHER_CODE Publisher
    ,i.PRODUCT_TYPE pt
    ,i.REPORTING_CATEGORY cat
    ,case
        when i.PUBLISHING_GROUP in('BAR-ENT','BAR-ART','BAR-FWN','BAR-LIF','BAR-CHL') then 'BAR'
        else i.PUBLISHING_GROUP
    end pgrp
    ,i.ITEM_TITLE ISBN
    ,i.SHORT_TITLE title
    ,i.PRICE_AMOUNT price
    ,convert(char,coalesce(convert(varchar,i.AMORTIZATION_DATE,101),shdt.shipdate),101) pub
    ,case
        when i.AMORTIZATION_DATE is not null then year(i.AMORTIZATION_DATE)
        when substring(i.season,1,4) <> 'No S' then substring(i.season,1,4)
        else year(getdate())
    end [year]

FROM ebs.Item i
    left join (SELECT [ISBN],[SHIPDATE] FROM [CBQ2].[pm].[ItemInfo] ii WHERE ii.IMPRESSION = 1 and ii.SHIPDATE is not null) shdt on shdt.ISBN = i.ISBN 
'''

In [31]:
df_item = pd.read_sql_query(query_item,conn)

In [32]:
df_item['pub'] = df_item['pub'].astype('datetime64[ns]')

df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65661 entries, 0 to 65660
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Publisher  65661 non-null  object        
 1   pt         65661 non-null  object        
 2   cat        65661 non-null  object        
 3   pgrp       65661 non-null  object        
 4   ISBN       65661 non-null  object        
 5   title      59831 non-null  object        
 6   price      49012 non-null  float64       
 7   pub        42231 non-null  datetime64[ns]
 8   year       65661 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 4.5+ MB


# Merge PO and Category Dataframes

In [33]:
def merge_cat_po(df_po,df_cat):
    df = pd.merge(df_po,df_cat,how = 'left',on = 'ISBN').fillna(0)
    return df

In [34]:
current_tc = round(sum(df_po['Total accepted cost']))
print(f'{current_tc:,.0f}')

588,745


In [35]:
# Combining the po dataframe and the category dataframe
df_cat_po = merge_cat_po(df_po_clean,df_cat_clean)

In [36]:
df_cat_po.info()

df_cat_po.rename(columns={'ASIN_x':'ASIN'},inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3378 entries, 0 to 3377
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ASIN_x               3378 non-null   object 
 1   ISBN                 3378 non-null   object 
 2   Accepted Quantity    3378 non-null   int64  
 3   Requested quantity   3378 non-null   int64  
 4   Total accepted cost  3378 non-null   float64
 5   Cost price           3378 non-null   float64
 6   ASIN_y               3378 non-null   object 
 7   Product Group        3378 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 237.5+ KB


In [37]:
sum(df_cat_po['Total accepted cost'])

588745.1999999984

In [38]:
df_cat_po['ASIN_y'].nunique()

3238

# Merge Tables

In [39]:
# combines both the df_po and df_item table and provides just the columns we want

def merge_clean_tables(df_po,df_item,df_avail):
    df_po['ISBN'] = df_po['ISBN'].str.zfill(13)
    df_avail['ISBN'] = df_avail['ISBN'].str.zfill(13)
    df_item['ISBN'] = df_item['ISBN'].str.zfill(13)
    df_po = df_po.loc[:,['ASIN','ISBN','Product Group','Accepted Quantity'\
                         ,'Requested quantity','Total accepted cost','Cost price']]
    df_item = df_item.loc[:,['ISBN','Publisher','pgrp','title','pub','price']]    
    #merge po and item tables
    df = pd.merge(df_po,df_item,how = 'left',left_on = 'ISBN',right_on = 'ISBN').fillna(0)
    # add variance column
    df['Delta'] = df['Accepted Quantity']-df['Requested quantity']
    #merge df_avail
    df = pd.merge(df,df_avail,how='left',on='ISBN')
    #Creating the "sales lost column"
    df['Lost Sales'] = (df['Requested quantity']*df['Cost price'])-df['Total accepted cost']
    #specify the order and columns we want in the final product
    df = df.loc[:,['ASIN','ISBN','title','Publisher','pgrp','pub'
                   ,'Product Group','Available To Sell','Reprint Due Date'
                   ,'Reprint Quantity','Requested quantity','Accepted Quantity'
                   ,'Delta','Total accepted cost','Lost Sales']]
    return df

In [40]:
df_combined = merge_clean_tables(df_cat_po,df_item,df_avail_clean)
print(df_combined.shape[0])

df_combined.shape

3378


(3378, 15)

# Basic Summaries

In [41]:
df_combo_grp = df_combined.groupby(by=['Publisher']).agg({'Total accepted cost':'sum'})
df_combo_grp

,Total accepted cost
Publisher,
AMMO Books,133.28
Chronicle,402803.44
Creative Company,3438.02
Galison,76489.98
Hardie Grant Publishing,40875.36
Laurence King,35628.48
Levine Querido,7891.76
PQ Blackwell,41.38
Paperblanks,7526.82


# Clean Merge

In [42]:
print(f"The PO table has {df_po_clean.shape[0]} rows.")
print(f"The combined table has {df_combined.shape[0]} rows.")

total_cost = sum(df_combined['Total accepted cost'])
print(f'The total "Total Cost" is {total_cost:,.0f}.')

The PO table has 3378 rows.
The combined table has 3378 rows.
The total "Total Cost" is 588,745.


In [43]:
df_combined[(df_combined['Publisher'] == 0)]

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales


In [44]:
# Removing the non-Chronicle titles from the list
df_combined_clean = df_combined[~(df_combined['Publisher'] == 0)]

In [45]:
sum(df_combined_clean['Total accepted cost'])

588745.1999999984

In [46]:
# I'm removing the ISBN's that don't actually belong to any CB or DP. See list below
df_combined_clean = df_combined[~(df_combined['Publisher'] == 0)]

# These orders are NOT for any of our titles.
print(df_combined[(df_combined['Publisher'] == 0)].shape[0])
df_combined[(df_combined['Publisher'] == 0)]

0


,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales


In [47]:
filt = (df_combined_clean['Publisher'] == 0)

df_pub_summary= df_combined_clean[~filt].groupby(by=['Publisher'])[['Requested quantity'\
                            ,'Accepted Quantity','Delta','Total accepted cost','Lost Sales']].sum().reset_index()
df_pub_summary.sort_values(by=['Total accepted cost'],ascending=False,inplace=True)
df_pub_summary['Total Requested Cost'] = df_pub_summary['Total accepted cost'] + df_pub_summary['Lost Sales']
df_pub_summary

,Publisher,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
1,Chronicle,61079,48057,-13022,402803.44,93903.680000,496707.120000
3,Galison,13378,10273,-3105,76489.98,25631.190000,102121.170000
4,Hardie Grant Publishing,7059,3216,-3843,40875.36,78627.340000,119502.700000
5,Laurence King,6153,3211,-2942,35628.48,31833.900000,67462.380000
11,Tourbillon,1947,1309,-638,13164.74,5843.380000,19008.120000
6,Levine Querido,1102,963,-139,7891.76,1050.180000,8941.940000
8,Paperblanks,1798,721,-1077,7526.82,9906.410000,17433.230000
2,Creative Company,1011,648,-363,3438.02,1835.400000,5273.420000
10,Sierra Club,81,81,0,743.12,0.000000,743.120000
0,AMMO Books,10,10,0,133.28,0.000000,133.280000


In [48]:
df_top_ordered_cb = df_combined_clean[df_combined_clean['Publisher']=='Chronicle'].sort_values(by=['Requested quantity'],ascending=False).head(20).reset_index(drop=True)

df_top_ordered_cb['Total Requested Cost'] = df_top_ordered_cb['Total accepted cost'] + df_top_ordered_cb['Lost Sales']

df_top_ordered_cb.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
0,1452179611,9781452179612,From Crook to Cook hc,Chronicle,FWN,2018-10-01 00:00:00,Books-Local Books,27307.0,2024-10-02,100000.0,3175,3142,-33,36070.16,378.84,36449.00
1,0811826023,9780811826020,Peek-A Who? bb,Chronicle,CHL,2000-02-01 00:00:00,0,-7287.0,2024-08-28,75000.0,2361,0,-2361,0.00,7555.20,7555.20
2,1797204327,9781797204321,Construction Site Gets a Fright! bb,Chronicle,CHL,2022-08-01 00:00:00,Books-Trade,18522.0,NaT,0.0,1983,1904,-79,11385.92,472.42,11858.34
3,1797213709,9781797213705,All About U.S.,Chronicle,CHL,2024-08-01 00:00:00,Books-Trade,15848.0,NaT,0.0,1346,1293,-53,11895.60,487.60,12383.20
4,0811864952,9780811864954,Ivy & Bean Secret Treasure Box,Chronicle,CHL,2008-10-01 00:00:00,Books-Trade,38066.0,NaT,0.0,1308,1284,-24,11812.80,220.80,12033.60


In [49]:
df_top_ordered_dp = df_combined_clean[~(df_combined_clean['Publisher']=='Chronicle')].sort_values(by=['Requested quantity'],ascending=False).head(20).reset_index(drop=True)

df_top_ordered_dp['Total Requested Cost'] = df_top_ordered_dp['Total accepted cost'] + df_top_ordered_dp['Lost Sales']

df_top_ordered_dp.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
0,1784883506,9781784883508,Resident Dog (Volume Two),Hardie Grant Publishing,QDP-BOOK,2020-11-01 00:00:00,Books-Trade,-14.0,2024-08-30,5000.0,1776,0,-1776,0.0,44932.80,44932.80
1,0735362149,9780735362147,Puz 500 Foil Winter Lights Foil,Galison,GAL,2020-08-01 00:00:00,Toys,733.0,2024-09-29,18000.0,880,880,0,6600.0,0.00,6600.00
2,0735375283,9780735375284,Puz 1000 Autumn In the Neighborhood,Galison,GAL,2022-06-01 00:00:00,Toys,15615.0,NaT,0.0,870,870,0,7830.0,0.00,7830.00
3,0735379343,9780735379343,BM Card Set Emergency Affirmations,Galison,GAL-BM,2023-07-01 00:00:00,Books-Trade,-402.0,2024-09-18,4000.0,863,0,-863,0.0,7145.64,7145.64
4,1743793456,9781743793459,This is Home,Hardie Grant Publishing,QDP-HGB,2018-04-01 00:00:00,Books-Trade,-1.0,2024-09-26,6000.0,780,0,-780,0.0,16146.00,16146.00


In [50]:
df_top_accepted_cb = df_combined_clean[df_combined_clean['Publisher']=='Chronicle'].sort_values(by=['Accepted Quantity'],ascending=False).head(20).reset_index(drop=True)

df_top_accepted_cb['Total Requested Cost'] = df_top_accepted_cb['Total accepted cost'] + df_top_accepted_cb['Lost Sales']

df_top_accepted_cb.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
0,1452179611,9781452179612,From Crook to Cook hc,Chronicle,FWN,2018-10-01 00:00:00,Books-Local Books,27307.0,2024-10-02,100000.0,3175,3142,-33,36070.16,378.84,36449.00
1,1797204327,9781797204321,Construction Site Gets a Fright! bb,Chronicle,CHL,2022-08-01 00:00:00,Books-Trade,18522.0,NaT,0.0,1983,1904,-79,11385.92,472.42,11858.34
2,1797213709,9781797213705,All About U.S.,Chronicle,CHL,2024-08-01 00:00:00,Books-Trade,15848.0,NaT,0.0,1346,1293,-53,11895.60,487.60,12383.20
3,0811864952,9780811864954,Ivy & Bean Secret Treasure Box,Chronicle,CHL,2008-10-01 00:00:00,Books-Trade,38066.0,NaT,0.0,1308,1284,-24,11812.80,220.80,12033.60
4,1452150184,9781452150185,This Is How We Do It,Chronicle,CHL,2017-05-01 00:00:00,0,2465.0,NaT,0.0,1255,1206,-49,10540.44,428.26,10968.70


In [51]:
df_top_accepted_dp = df_combined_clean[~(df_combined_clean['Publisher']=='Chronicle')].sort_values(by=['Accepted Quantity'],ascending=False).head(20).reset_index(drop=True)

df_top_accepted_dp['Total Requested Cost'] = df_top_accepted_dp['Total accepted cost'] + df_top_accepted_dp['Lost Sales']

df_top_accepted_dp.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
0,0735362149,9780735362147,Puz 500 Foil Winter Lights Foil,Galison,GAL,2020-08-01 00:00:00,Toys,733.0,2024-09-29,18000.0,880,880,0,6600.00,0.00,6600.00
1,0735375283,9780735375284,Puz 1000 Autumn In the Neighborhood,Galison,GAL,2022-06-01 00:00:00,Toys,15615.0,NaT,0.0,870,870,0,7830.00,0.00,7830.00
2,1784881112,9781784881115,365 Days of Art,Hardie Grant Publishing,QDP-BOOK,2017-10-01 00:00:00,Books-Trade,10310.0,NaT,0.0,485,457,-28,4835.06,296.24,5131.30
3,1646140001,9781646140008,Everything Sad Is Untrue,Levine Querido,LQ-AL,2020-08-01 00:00:00,Books-Trade,1023.0,NaT,0.0,401,379,-22,3312.46,192.28,3504.74
4,0735379483,9780735379480,BM Card Game Set Poorly Explained Movies,Galison,GAL-BM,2023-07-01 00:00:00,Toys,-1599.0,2024-10-25,6000.0,362,362,0,3258.00,0.00,3258.00


In [52]:
# dataframe of titles that were requested but not accepted
df_lost_sales = df_combined_clean.sort_values(by=['Lost Sales'],ascending=False).head(20).reset_index(drop=True)

df_lost_sales['Total Requested Cost'] = df_lost_sales['Total accepted cost'] + df_lost_sales['Lost Sales']

In [53]:
df_lost_sales.head()

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales,Total Requested Cost
0,1784883506,9781784883508,Resident Dog (Volume Two),Hardie Grant Publishing,QDP-BOOK,2020-11-01 00:00:00,Books-Trade,-14.0,2024-08-30,5000.0,1776,0,-1776,0.0,44932.8,44932.8
1,1743793456,9781743793459,This is Home,Hardie Grant Publishing,QDP-HGB,2018-04-01 00:00:00,Books-Trade,-1.0,2024-09-26,6000.0,780,0,-780,0.0,16146.0,16146.0
2,0811876659,9780811876650,Ivy & Bean Boxed Set 2 (Bk 4-6),Chronicle,CHL,2010-09-01 00:00:00,0,-14.0,2024-08-23,17500.0,963,0,-963,0.0,8859.6,8859.6
3,179721389X,9781797213897,"Henry, Like Always hc",Chronicle,CHL,2023-03-01 00:00:00,Books-Trade,-1959.0,2024-08-28,7500.0,1147,0,-1147,0.0,7914.3,7914.3
4,0811826023,9780811826020,Peek-A Who? bb,Chronicle,CHL,2000-02-01 00:00:00,0,-7287.0,2024-08-28,75000.0,2361,0,-2361,0.0,7555.2,7555.2


In [54]:
# Creating tables to summarize sales by product group

filt = (df_combined_clean['Publisher']=='Chronicle')

cb_product_group = df_combined_clean[filt].groupby(['Publisher','Product Group'])['Total accepted cost'].sum().reset_index()
dp_product_group = df_combined_clean[~filt].groupby(['Publisher','Product Group'])['Total accepted cost'].sum().reset_index()

In [55]:
total_cost_combined_clean = df_combined_clean['Total accepted cost'].sum()
cb_total_cost_combined_clean = cb_product_group['Total accepted cost'].sum()
dp_total_cost_combined_clean = dp_product_group['Total accepted cost'].sum()

print(f'Total should be {total_cost_combined_clean:,.0f}.')
print()
print(f'CB total should be {cb_total_cost_combined_clean:,.0f}.')
print(f'DP total should be {dp_total_cost_combined_clean:,.0f}.')

print(f'CB + DP total should be {cb_total_cost_combined_clean + dp_total_cost_combined_clean:,.0f}.')

Total should be 588,745.

CB total should be 402,803.
DP total should be 185,942.
CB + DP total should be 588,745.


In [56]:
print(cb_product_group.head())

df_combined_clean.loc[df_combined_clean['Product Group'] == 'Digital_Ebook_Purchase']

   Publisher           Product Group  Total accepted cost
0  Chronicle                       0             59937.63
1  Chronicle                    Baby              5289.96
2  Chronicle       Books-Local Books             36070.16
3  Chronicle             Books-Trade            272349.65
4  Chronicle  Digital_Ebook_Purchase               148.58


,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Requested quantity,Accepted Quantity,Delta,Total accepted cost,Lost Sales
2756,1797209655,9781797209654,Books Make Good Friends,Chronicle,CHL,2023-11-01 00:00:00,Digital_Ebook_Purchase,1372.0,2024-10-30,3000.0,17,17,0,148.58,0.0


In [57]:
# Used to provide a table for the tables that were rejected
reject_rows = df_combined.loc[df_combined['Publisher'] == 0]

In [58]:
path = r'G:\SALES\Amazon\PURCHASE ORDERS\atelier\po_analysis\amazon_order_py_dump.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_pub_summary.to_excel(writer,sheet_name='pub_summary',index = False)
df_top_ordered_cb.to_excel(writer,sheet_name='ordered_summary_cb',index = False)
df_top_ordered_dp.to_excel(writer,sheet_name='ordered_summary_dp',index = False)
df_top_accepted_cb.to_excel(writer,sheet_name='accepted_summary_cb',index=False)
df_top_accepted_dp.to_excel(writer,sheet_name='accepted_summary_dp',index=False)
df_lost_sales.to_excel(writer,sheet_name='lost_sales_summary',index=False)
cb_product_group.to_excel(writer,sheet_name='cb_group_summary',index=False)
dp_product_group.to_excel(writer,sheet_name='dp_group_summary',index=False)
reject_rows.to_excel(writer,sheet_name='reject_summary',index=False)

writer.save()

# Creating df's for DPs

In [59]:
df_combined.pub.replace(0,pd.NaT,inplace = True)
df_combined['pub'] = pd.to_datetime(df_combined['pub'])

In [60]:
# df_temp.groupby(['ASIN','ISBN','title','pub','Reprint Due Date','Reprint Quantity','pgrp'])['Quantity Requested'].count()
df_combined_dp = df_combined.copy()
df_combined_dp.dtypes

ASIN                           object
ISBN                           object
title                          object
Publisher                      object
pgrp                           object
pub                    datetime64[ns]
Product Group                  object
Available To Sell             float64
Reprint Due Date       datetime64[ns]
Reprint Quantity              float64
Requested quantity              int64
Accepted Quantity               int64
Delta                           int64
Total accepted cost           float64
Lost Sales                    float64
dtype: object

In [61]:

df_combined_dp['pub'] = df_combined_dp.loc[:,'pub'].dt.strftime('%Y-%m-%d')
df_combined_dp['pub'].fillna("-",inplace=True)

# df_combined_dp['Reprint Due Date'] = df_combined_dp.loc[:,'Reprint Due Date'].dt.strftime('%Y-%m-%d')
df_combined_dp['Reprint Due Date'].fillna("-",inplace=True)
df_combined_dp['Available To Sell'].fillna(0,inplace=True)
df_combined_dp['Reprint Quantity'].fillna(0,inplace=True)

df_combined_dp.isnull().sum()

ASIN                   0
ISBN                   0
title                  0
Publisher              0
pgrp                   0
pub                    0
Product Group          0
Available To Sell      0
Reprint Due Date       0
Reprint Quantity       0
Requested quantity     0
Accepted Quantity      0
Delta                  0
Total accepted cost    0
Lost Sales             0
dtype: int64

In [62]:
# Function to create dp list without 0 and "Chronicle"
def create_dp_list(col):
    dp_list = list(col)
    if 0 in dp_list:
        dp_list.remove(0)
    if 'Chronicle' in dp_list:
        dp_list.remove('Chronicle')
    return dp_list

In [63]:
# Create DP List.
dp_list = create_dp_list(df_combined_dp.Publisher.unique())

remove_list = ['PQ Blackwell','San Francisco Art Institute','Driscolls']

dp_list = [i for i in dp_list if i not in remove_list]

print(dp_list)

['Galison', 'Laurence King', 'Creative Company', 'Paperblanks', 'Sierra Club', 'AMMO Books', 'Levine Querido', 'Hardie Grant Publishing', 'Tourbillon', 'Princeton']


In [64]:
# filter the table to single dp
def filter_dp(df,dp):
    df_new = df.loc[(df['Publisher']==dp)]
    return df_new

In [65]:
# creates a pub by pgrp view
def dp_by_pgrp(df):
    df_dp = df.groupby(
        ['pgrp']).agg(
        Sum_of_Total_Cost = pd.NamedAgg(column='Total accepted cost',aggfunc=sum))
    return df_dp

In [66]:
# top twenty
def dp_top20(df,col):
    df_dp = df.groupby(
        ['ASIN','ISBN','title','pub','Reprint Due Date','Reprint Quantity','pgrp']).agg(
        Quantity_Requested = pd.NamedAgg(column='Requested quantity',aggfunc=sum)
        ,Quantity_Accepted = pd.NamedAgg(column='Accepted Quantity',aggfunc=sum)
        ,Total_Cost = pd.NamedAgg(column='Total accepted cost',aggfunc=sum))
    df_dp = df_dp.sort_values(by=col,ascending = False).head(20)
    
    return df_dp

In [67]:
# Creates dictionary and runs all three dataframes for each publisher.
dp_dict = {}

for dp in dp_list:
    df_temp = filter_dp(df_combined_dp,dp)
    dp_dict[dp] = list()
    dp_dict[dp].append(dp_by_pgrp(df_temp).reset_index(drop=False))
    dp_dict[dp].append(dp_top20(df_temp,col = 'Quantity_Requested').reset_index(drop=False))
    dp_dict[dp].append(dp_top20(df_temp,col = 'Quantity_Accepted').reset_index(drop=False))
    dp_dict[dp].append(df_combined_dp.loc[df_combined_dp.Publisher==dp])

## This is used for creating the emails

In [68]:
path = r'G:\SALES\Amazon\PURCHASE ORDERS\atelier\po_analysis\amazon_order_dp_dump.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

for dp in list(dp_dict.keys()):
    dp_dict[dp][0].to_excel(writer,sheet_name=f'{dp}_PGRP',index = False)
    dp_dict[dp][1].to_excel(writer,sheet_name=f'{dp}_QtyRqt',index = False)
    dp_dict[dp][2].to_excel(writer,sheet_name=f'{dp}_QtyAcpt',index = False)
    dp_dict[dp][3].to_excel(writer,sheet_name=f'{dp}_Detail',index = False)
writer.save()

In [69]:
import os
notebook_path = os.getcwd()
print(notebook_path)

C:\Users\rbh\Documents\Python Scripts\amazon


## This is used for creating the the file that's saved off with some summary/pivot views

In [70]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
cb_product_group     DataFrame       Publisher           Pr<...>Toys             26102.86
df_asin              DataFrame                    ISBN     <...>n[46188 rows x 2 columns]
df_asin_clean        DataFrame                    ISBN     <...>n[46188 rows x 2 columns]
df_avail             DataFrame                    ISBN On S<...>n[39220 rows x 5 columns]
df_avail_clean       DataFrame                    ISBN On S<...>n[39160 rows x 5 columns]
df_cat               DataFrame                 ASIN        <...>n[29225 rows x 4 columns]
df_cat_clean         DataFrame                 ASIN        <...>n[29093 rows x 3 columns]
df_cat_po            DataFrame                ASIN         <...>\n[3378 rows x 8 columns]
df_combined          DataFrame                ASIN         <...>n[3378 rows x 15 columns]
df_combined_clean    DataFrame                ASIN         <...>n[3378 rows x 15 columns]
df_combined_

In [71]:
# Pivoting the PO doc from Amazon Vender Central
df_pivot1 = df_combined_clean.groupby(['ASIN','ISBN','title','Publisher','pgrp'])[['Requested quantity'\
                                    , 'Accepted Quantity','Total accepted cost']].sum().reset_index()

df_pivot1.head()

,ASIN,ISBN,title,Publisher,pgrp,Requested quantity,Accepted Quantity,Total accepted cost
0,0473173891,9780473173890,Strip Search,PQ Blackwell,PQB,1,1,41.38
1,0735329141,9780735329140,Ntcd Keepsake Box Van Gogh Floral Collection,Galison,GAL,4,4,27.60
2,0735330972,9780735330979,Puz 12 Pouch On the Farm,Galison,MUD,18,18,99.00
3,073533112X,9780735331129,Puz 12 Pouch Under the Sea,Galison,MUD,36,36,198.00
4,073533417X,9780735334175,Puz 25 Jumbo At the Zoo,Galison,MUD,4,4,30.00


In [72]:
# Merging in the available document
list_avail = ['ISBN','On Sale Date','Reprint Due Date','Reprint Quantity']

df_pivot1 = pd.merge(df_pivot1,df_avail_clean[list_avail],on='ISBN',how='left')

In [73]:
# there used to be Reprint Quantities without "Reprint Due Date" ... this doesn't seem to be the problem any more.
df_pivot1.loc[(df_pivot1['Reprint Quantity'] > 0) & df_pivot1['Reprint Due Date'].isna(), 'Reprint Quantity'] = 0

In [74]:
print(df_pivot1.shape)
df_pivot1.head()

(3378, 11)


,ASIN,ISBN,title,Publisher,pgrp,Requested quantity,Accepted Quantity,Total accepted cost,On Sale Date,Reprint Due Date,Reprint Quantity
0,0473173891,9780473173890,Strip Search,PQ Blackwell,PQB,1,1,41.38,NaT,NaT,0.0
1,0735329141,9780735329140,Ntcd Keepsake Box Van Gogh Floral Collection,Galison,GAL,4,4,27.60,NaT,NaT,0.0
2,0735330972,9780735330979,Puz 12 Pouch On the Farm,Galison,MUD,18,18,99.00,2011-05-01,2024-09-17,5000.0
3,073533112X,9780735331129,Puz 12 Pouch Under the Sea,Galison,MUD,36,36,198.00,2011-05-01,2024-11-04,5000.0
4,073533417X,9780735334175,Puz 25 Jumbo At the Zoo,Galison,MUD,4,4,30.00,NaT,NaT,0.0


In [75]:
df_pivot2 = df_pivot1.groupby(['Publisher'])[['Total accepted cost']].sum().reset_index()

In [76]:
df_pivot3 = df_pivot1.groupby(['Publisher','pgrp'])[['Total accepted cost']].sum().reset_index()

In [77]:
df_pivot3.head()

,Publisher,pgrp,Total accepted cost
0,AMMO Books,AMMO,133.28
1,Chronicle,ART,35480.32
2,Chronicle,BAR,189.59
3,Chronicle,CCB,2004.42
4,Chronicle,CHL,165137.34


In [78]:
today = datetime.date.today()
current_year = today.year

# Find the prior Sunday and Monday
weekday = today.weekday()
if weekday == 0:  # If today is Monday, consider prior Sunday and Monday
    prior_sunday = today - datetime.timedelta(days=weekday+1)
    prior_monday = today
else:  # For other weekdays, find the prior Sunday and Monday normally
    prior_sunday = today - datetime.timedelta(days=weekday+1)
    prior_monday = today - datetime.timedelta(days=weekday+0)

# Generate the file name
file_name = f"{current_year}_{prior_sunday.strftime('%m_%d_%A')}_{prior_monday.strftime('%m_%d_%A')}"

print(f"This week's file name is: << {file_name} >>")

This week's file name is: << 2024_08_11_Sunday_08_12_Monday >>


In [79]:
path2 = rf'G:\SALES\Amazon\PURCHASE ORDERS\2024\{file_name}.xlsx'
writer = pd.ExcelWriter(path2, engine='xlsxwriter')

df_po.to_excel(writer,sheet_name=f'PurchaseOrderItems_raw',index = False)
df_combined_clean.to_excel(writer,sheet_name=f'POItems_Aggregated',index = False)
df_pivot1.to_excel(writer,sheet_name=f'Pivot_POs',index = False)
df_pivot2.to_excel(writer,sheet_name=f'Pivot_by_Publisher',index = False)
df_pivot3.to_excel(writer,sheet_name=f'Pivot_by_Pub_Pgrp',index = False)

writer.save()

PermissionError: [Errno 13] Permission denied: 'G:\\SALES\\Amazon\\PURCHASE ORDERS\\2024\\2024_08_11_Sunday_08_12_Monday.xlsx'

In [ ]:
end_time = datetime.datetime.now()
print(f"Update completed\nData tab script took {end_time-start} to run.")